# Workshop: Performance

#### Descripition
This workshop aims to present how to efficiently code in Python.

**Course**: INF8111 - Fouille de données (Fall 2020)

**Author**: Quentin Fournier

**Inspired by**: [Towards Data Science](https://towardsdatascience.com/ten-tricks-to-speed-up-your-python-codes-c38abdb89f18), [Wiki](https://wiki.python.org/moin/PythonSpeed/PerformanceTips#Avoiding_dots...), [Cookbook on Numpy](https://ipython-books.github.io/45-understanding-the-internals-of-numpy-to-avoid-unnecessary-array-copying/)

# Exemple 1 - Loop, List Comprehension, and Built-in Function

In [1]:
%%timeit -n10

# Create a list with all integer from 0 to 999 999 using a for loop
x = []
for i in range(1000000):
  x.append(i)

10 loops, best of 3: 103 ms per loop


In [2]:
%%timeit -n10

# List comprehension (elegant and faster)
x = [i for i in range(1000000)]

10 loops, best of 3: 65.5 ms per loop


In [3]:
%%timeit -n10

# Built-in function are always better
x = list(range(1000000))

10 loops, best of 3: 38.9 ms per loop


# Exemple 2 - Loop, List Comprehension, and Function Call

In [4]:
data = list(range(100000))

In [5]:
%%timeit -n10

# Function that works on an integer
def fct(x):
  return x ** 2

# For loop that calls the function multiple times
y = []
for x in data:
  y.append(fct(x))

10 loops, best of 3: 39.1 ms per loop


In [6]:
%%timeit -n10

# Function that works on an integer
def fct(x):
  return x ** 2

# List comprehension that calls the function multiple times
y = [fct(x) for x in data]

10 loops, best of 3: 36.8 ms per loop


In [7]:
%%timeit -n10

# Function that works on a list using a for loop
def fct(data):
  y = []
  for x in data:
    y.append(x ** 2)
  return y

# The function is called only once
y = fct(data)

10 loops, best of 3: 32.2 ms per loop


In [8]:
%%timeit -n10

# Function that works on a list using list comprehension
def fct(data):
  return [x ** 2 for x in data]

# The function is called only once
y = fct(data)

10 loops, best of 3: 28.2 ms per loop


In [9]:
# List comprehension (the function was not useful)
%timeit -n10 y = [x ** 2 for x in data]

10 loops, best of 3: 28.3 ms per loop


# Exemple 3 - Enumerate, Zip, and Map

In [10]:
data = ['a', 'b', 'c']
value = [12, 643, 123, 687, 213]

In [11]:
# Print data with the index (manually) 
i = 0
for d in data:
  print('({}) {}'.format(i, d))
  i += 1

(0) a
(1) b
(2) c


In [12]:
# Print data with the index (enumerate) 
for i, d in enumerate(data):
  print('({}) {}'.format(i, d))

(0) a
(1) b
(2) c


In [13]:
# Go through two or more lists at the same time
for i, (d, v) in enumerate(zip(data, value)):
  print('({}) {} : {}'.format(i, d, v))

(0) a : 12
(1) b : 643
(2) c : 123


In [14]:
# Matrix with 3 lines, and 2 columns
matrix = [[1, 2], [3, 4], [5, 6]]
print(matrix)

[[1, 2], [3, 4], [5, 6]]


In [15]:
# Transpose the matrix in pure Pyhton
matrix_transpose = list(map(list, zip(*matrix)))
print(matrix_transpose)

[[1, 3, 5], [2, 4, 6]]


In [16]:
# Step-by-step
print('matrix:')
print(matrix)
print('\nUnpacked matrix *matrix:')
print(*matrix)
print(matrix[0], matrix[1], matrix[2])
print('\nZiped unpacked matrix zip(*matrix):')
print(zip(*matrix))
print('\nWe need to convert the object in a list list(zip(*matrix)):')
print(list(zip(*matrix)))
print('\nZiped unpacked matrix (list(map(list, zip(*matrix)))):')
print(list(map(list, zip(*matrix))))

matrix:
[[1, 2], [3, 4], [5, 6]]

Unpacked matrix *matrix:
[1, 2] [3, 4] [5, 6]
[1, 2] [3, 4] [5, 6]

Ziped unpacked matrix zip(*matrix):

We need to convert the object in a list list(zip(*matrix)):
[(1, 3, 5), (2, 4, 6)]

Ziped unpacked matrix (list(map(list, zip(*matrix)))):
[[1, 3, 5], [2, 4, 6]]


# Exemple 4 - Generator

In [17]:
import itertools

# Function that returns the list of squared values
def function(data):
  return [x**2 for x in data]

# Function that yields the squared values
def generator(data):
  for x in data:
    yield x**2

In [18]:
# a is instanciated in memory
data = range(1000000)
%timeit -n10 a = function(data)

10 loops, best of 3: 298 ms per loop


In [19]:
# b is NOT instanciated in memory, so almost no time required
data = range(1000000)
%timeit -n10 b = generator(data)

10 loops, best of 3: 224 ns per loop


In [20]:
# b is not a list but a generator
b = generator(data)
print(b)

<generator object generator at 0x7f5353c976d0>


In [21]:
%%timeit -n10

# The list a is computed entirely, even if 5 values are needed
a = function(data)
for x in a[:5]:
  print(x, end='\r')

10 loops, best of 3: 308 ms per loop


In [22]:
%%timeit -n100

# With a generator, values are computed when needed
b = generator(data)
i = 0
for i, x in enumerate(b):
  if i > 4:
    break
  print(x, end='\r')

The slowest run took 4.46 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 103 µs per loop


In [23]:
%%timeit -n100

# A better way to manipulate a generator is with the library itertools
b = generator(data)
for x in itertools.islice(b, 5):
  print(x, end='\r')

100 loops, best of 3: 426 µs per loop


In [24]:
# Like list comprehension, generator may be obtain in one line with ( ) instead of [ ]
b = (x**2 for x in data)

# Exemple 5 - Advanced Concept

In [25]:
import string
import random

In [26]:
%%timeit -n10

# Generate random letters, capitalize them, then add them to a list
newlist = []
for _ in range(100000):
    newlist.append(random.choice(string.ascii_letters).upper())

10 loops, best of 3: 90.2 ms per loop


In [27]:
%%timeit -n10

newlist = []

# Redefined functions to remove the dot
upper = str.upper
choice = random.choice
alph = string.ascii_letters 
append = newlist.append

# For loop without dots
for _ in range(100000):
  append(upper(choice(alph)))

10 loops, best of 3: 75.9 ms per loop


In [36]:
%%timeit -n10

# Local variables are less expensive that global ones
def up():
  newlist = []
  upper = str.upper
  choice = random.choice
  alph = string.ascii_letters 
  append = newlist.append
  for _ in range(100000):
    append(upper(choice(alph)))
  newlist = []

newlist = up()

10 loops, best of 3: 75.2 ms per loop


In [29]:
%%timeit -n10

def up():
  upper = str.upper
  choice = random.choice
  alph = string.ascii_letters
  return [upper(choice(alph)) for _ in range(100000)]

newlist = up()

10 loops, best of 3: 75.1 ms per loop


In [30]:
# Sometimes, list comprehension are slower
%timeit -n10 newlist = [random.choice(string.ascii_letters).upper() for _ in range(100000)]

10 loops, best of 3: 80.2 ms per loop


# Exemple 6 - Numpy, In-Place, Contiguous

In [31]:
import numpy as np

In [32]:
%%timeit -n10

# Create a new array in memory to store a + 2, even if it has the same name
a = np.zeros(10000000)
a = a + 2

10 loops, best of 3: 34.1 ms per loop


In [33]:
%%timeit -n10

# In-place operation (modify the object in memory)
a = np.zeros(10000000)
a += 2

10 loops, best of 3: 23.9 ms per loop


In [34]:
%%timeit -n10

# Matrix of size 10000 x 10000
a = np.zeros((10000, 10000))
# Reshape the matrix to be of size (1, 100 000 000)
b = a.reshape((1, -1))

10 loops, best of 3: 182 ms per loop


In [35]:
%%timeit

a = np.zeros((10000, 10000))
# A is transposed, but remains the same in memory (not contiguous anymore)
b = a.T.reshape((1, -1))

1 loop, best of 3: 3.12 s per loop
